## Hangman Game
Write an algorithm that plays the game of Hangman. 

If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

I use a training set of approximately 250,000 dictionary words. My algorithm will improve the successful rate to 69.57%


In [1]:
import random
import string
import time
import re
import collections

In [2]:
from collections import *
from math import *
from sklearn.model_selection import train_test_split
from itertools import combinations

### Language Model to train the dictionary
1. Get the probablity of letter appearance in differenet combination of gram
2. Hyper-parameter order decide the length of gram
        
        ## add different combination of gram into the trained language model 
        ## e.g. apple with order = 3, grams = "app","ppl","ple"
        ## for gram "app" below will be added into language model with the appearance of letter probability
        ## ap_, _pp, a_p
        ## a__, _p_, __p

In [3]:
def update_counter(lm, data, order):
    for i in range(len(data)-order):
        history, char = data[i:i+order], data[i+order]
        if history.find(' ') == -1 and char != ' ':
            lm[history][char]+=1
    return lm

def train_char_lm(dic, order=3):
    lm = defaultdict(Counter)
    data = dic
    # calculate the probability of letter appearnce in a specified gram (length decided by order)
    for i in range(len(data)-order):
        gram = data[i:i+order]
        histories = []
        chars = []
        if gram.find(' ') == -1 and gram.find('_') == -1:
            replace_space = order - 1
            while(replace_space>0):
                gram_list = list(gram)
                comb = combinations(range(len(gram)), replace_space)
                for i in list(comb):
                    keeped_letter = [x if idx not in list(i) else '_' for idx, x in enumerate(gram_list)]
                    replaced_letter = [x for idx, x in enumerate(gram_list) if idx in list(i)]
                    histories += ["".join(keeped_letter)]
                    chars += [Counter("".join(replaced_letter)).most_common()[0][0]]               
                replace_space -= 1
        for history, char in zip(histories, chars):
            if history.find(' ') == -1 and char != ' ':
                lm[history][char]+=1
    
    def normalize(counter):
        s = float(sum(counter.values()))
        return [(c,cnt/s) for c,cnt in counter.items()]
    outlm = {hist:normalize(chars) for hist, chars in lm.items()}
    return outlm

In [4]:
def check_guessed(sorted_letter_count, guessed_letters):
    flag = False
    for letter, instance_prob in sorted_letter_count:
        if letter not in guessed_letters and letter is not '_':
            flag = True
            break
    if flag:
        return (letter, instance_prob)
    else:
        return

def update_candidate(candidate_list, curr_guess):
    if curr_guess is not None:
        if curr_guess[0] in candidate_list:
            candidate_list[curr_guess[0]] += curr_guess[1]
        else:
            candidate_list[curr_guess[0]] = curr_guess[1]
    return candidate_list

def generate_letter(full_dict, guess, lm, d, guessed_letters, order = 3):
    candidate_list = {} 
    print (guess)
     ## the first time guess is based on probablity of all words with the length of target word
    if len(set(guess))== 1 and guess[0] == '_':
        curr_guess = check_guessed(collections.Counter("".join(d[len(guess)])).most_common(), guessed_letters)
        candidate_list = update_candidate(candidate_list, curr_guess)
    
    ## get the gram of current word and get the probablity of letters from language model
    for i in range(len(guess)):
        stem = guess[i:i+order]
        if stem in lm:
            curr_guess = check_guessed(sorted(lm[stem], key=lambda item:item[1], reverse=True), guessed_letters)
            candidate_list = update_candidate(candidate_list, curr_guess)
            
    ## if the gram is not in the language model, then use the default probability of all words with the length of target word
    if (len(candidate_list) == 0):
        curr_guess = check_guessed(collections.Counter("".join(d[len(guess)])).most_common(), guessed_letters)
        candidate_list = update_candidate(candidate_list, curr_guess)
        
    letter = max(candidate_list, key=lambda k: candidate_list[k])
    return letter

def play(answer, lm,  d, order, nTrials=6):
    guess = "_ " * int(len(answer)/2)
    guess_clean = guess[::2].replace(" ", "")
    full_dict = full_dictionary
    guessed_letters = []
    errors = 0
    count = 0
    flag = False
    while(errors < nTrials):
        c = generate_letter(full_dict, guess_clean, lm, d, guessed_letters, order)
        guessed_letters += [c]
        if answer.find(c)!=-1:
            idx = [pos for pos, char in enumerate(answer) if char == c]
            for j in idx:
                guess = '%s%s%s'%(guess[:j],c,guess[j+1:])
        else:
            errors += 1
        print ("-------------")   
        print (count, errors, c, ':  ', guess_clean)
        print ("-------------")
        guess_clean = guess[::2]
        if guess_clean.find('_') == -1:
            flag = True
            break
        count += 1
    return guess, flag

### Load the dictionary

In [5]:
def build_dictionary(dictionary_file_location):
    text_file = open(dictionary_file_location,"r")
    full_dictionary = text_file.read().splitlines()
    text_file.close()
    return full_dictionary

full_dictionary_location = "words_250000_train.txt"
full_dictionary = build_dictionary(full_dictionary_location)

### Train the Language Model

In [6]:
full_dic, answers = train_test_split(full_dictionary, test_size = 0.0001)
lm = train_char_lm(" ".join(full_dic), 5)
d=defaultdict(list)
for word in full_dic:
    d[len(word)].append(word)

### Start the Game

In [7]:
N = len(answers)
success = 0
for answer in answers:
    answer = " ".join(answer) + " "
    res, flag = play(answer, lm, d, order = 5)
    if flag:
        success += 1
        print ("success!, the answer is " + res)
    else:
        print("failed!, the answer is " + answer + " the guess is " + res)

_______
-------------
(0, 0, 'e', ':  ', '_______')
-------------
____e__
-------------
(1, 1, 'i', ':  ', '____e__')
-------------
____e__
-------------
(2, 2, 'a', ':  ', '____e__')
-------------
____e__
-------------
(3, 2, 's', ':  ', '____e__')
-------------
____e_s
-------------
(4, 2, 'r', ':  ', '____e_s')
-------------
____ers
-------------
(5, 3, 'p', ':  ', '____ers')
-------------
____ers
-------------
(6, 3, 'o', ':  ', '____ers')
-------------
_o__ers
-------------
(7, 3, 'h', ':  ', '_o__ers')
-------------
ho__ers
-------------
(8, 4, 'l', ':  ', 'ho__ers')
-------------
ho__ers
-------------
(9, 5, 'd', ':  ', 'ho__ers')
-------------
ho__ers
-------------
(10, 6, 't', ':  ', 'ho__ers')
-------------
failed!, the answer is h o c k e r s  the guess is h o _ _ e r s 
__________
-------------
(0, 0, 'e', ':  ', '__________')
-------------
e_________
-------------
(1, 0, 'i', ':  ', 'e_________')
-------------
e____i____
-------------
(2, 1, 'a', ':  ', 'e____i____')
-----

-------------
(0, 0, 'e', ':  ', '_________')
-------------
____e__e_
-------------
(1, 1, 'i', ':  ', '____e__e_')
-------------
____e__e_
-------------
(2, 2, 'a', ':  ', '____e__e_')
-------------
____e__e_
-------------
(3, 2, 's', ':  ', '____e__e_')
-------------
____e__es
-------------
(4, 2, 'r', ':  ', '____e__es')
-------------
____er_es
-------------
(5, 3, 'p', ':  ', '____er_es')
-------------
____er_es
-------------
(6, 4, 'l', ':  ', '____er_es')
-------------
____er_es
-------------
(7, 5, 't', ':  ', '____er_es')
-------------
____er_es
-------------
(8, 6, 'h', ':  ', '____er_es')
-------------
failed!, the answer is c o m m e r c e s  the guess is _ _ _ _ e r _ e s 
____________
-------------
(0, 1, 'e', ':  ', '____________')
-------------
____________
-------------
(1, 1, 'i', ':  ', '____________')
-------------
___i_i___i__
-------------
(2, 1, 'a', ':  ', '___i_i___i__')
-------------
_a_i_i_a_i__
-------------
(3, 1, 'c', ':  ', '_a_i_i_a_i__')
-------------
_a

In [8]:
acc = success/(N*1.0)*100
print ("success rate is %0.2f%%"%acc)

success rate is 47.83%
